In [4]:
import sys
sys.path.append("../")

In [5]:
from field_keypoints_detection import load_tfrecords

import matplotlib.pyplot as plt
import cv2
import numpy as np


In [7]:
train_dataset = load_tfrecords("data/train.tfrecord") # TODO: change to a real file
val_dataset = load_tfrecords("data/val.tfrecord") # TODO: change to a real file
val_dataset = load_tfrecords("data/test.tfrecord") # TODO: change to a real file

# Plotting Data Example

In [8]:
def plot_examples(parsed_and_decoded_dataset, num_examples=1):

  for image, keypoints in parsed_and_decoded_dataset.take(num_examples):
    plt.figure(figsize=(12,7))
    plt.imshow(image.numpy())
    plt.axis(False)


    # Reshape the keypoints array to have shape (num_keypoints, 3)
    keypoints = keypoints.numpy()
    keypoints = keypoints.reshape(-1, 3)

    for i, keypoint in enumerate(keypoints):
      x, y, visibility = keypoint
      if visibility == 2:  # Only plot visible keypoints
        plt.scatter(x, y, s=50, c='red', marker='o')
        plt.text(x, y, str(i), fontsize=12, color='blue', ha='right', va='bottom')


    plt.show()

In [9]:
plot_examples(train_dataset)

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: data/train.tfrecord : The system cannot find the file specified.
; No such file or directory [Op:IteratorGetNext] name: 

# Calculating the Visibility of Each Keypoint

In [10]:
def calculate_visibility(dataset):
    keypoint_visibility_count = [0] * 32  # Assuming 32 keypoints
    total_examples = 0

    for image, keypoints in dataset:
        keypoints = keypoints.numpy().reshape(-1, 3)  # Reshape to (num_keypoints, 3)
        for i, keypoint in enumerate(keypoints):
            if keypoint[2] == 2:  # Visibility indicator
                keypoint_visibility_count[i] += 1
        total_examples += 1

    visibility_percentage = [(count / total_examples) * 100 for count in keypoint_visibility_count]
    return visibility_percentage

In [22]:
all_dataset = train_dataset.concatenate(val_dataset).concatenate(test_dataset)
visibility_percentage = calculate_visibility(all_dataset)

top_down_keypoints = np.array([
  [98, 19], [98, 268], [98, 460], [98, 773], [98, 964], [98, 1199], # 0-5
  [188, 460], [188, 773], # 6-7
  [292, 616], # 8
  [380, 268], [380, 481], [380, 752], [380, 964], # 9-12
  [1017, 19], [1017, 460], [1017, 773], [1017, 1199], # 13-16
  [1641, 268], [1641, 481], [1641, 752], [1641, 964], # 17-20
  [1733, 616], # 21
  [1833, 453], [1833, 767], # 22-23
  [1923, 19], [1923, 268], [1923, 460], [1923, 773], [1923, 964], [1923, 1199], # 24-29
  [861, 616], [1174, 616] # 30-31
])

template = cv2.imread('2d_football_pitch.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12,7))
plt.title("Percentage of frames in which points are visible")
plt.axis(False)
plt.imshow(template)

for i, percentage in enumerate(visibility_percentage):
  plt.scatter(top_down_keypoints[i][0], top_down_keypoints[i][1], s=50, c='red', marker='o')
  plt.text(top_down_keypoints[i][0], top_down_keypoints[i][1], f'{round(percentage)}%', fontsize=10, color='blue', ha='right', va='bottom')

plt.savefig('../output_videos/percentage_of_visible_frames_for_each_point.png')
plt.show()

NameError: name 'train_dataset' is not defined